In [1]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Reading data

In [2]:
start_date = "2015-10-01"
end_date = "2024-10-31"

# Get the data
cpi = pdr.get_data_fred("MEDCPIM158SFRBCLE", start=start_date, end=end_date) # Change in CPI
unrate = pdr.get_data_fred("UNRATE", start=start_date, end=end_date) # Unemployment Rate
dff = pdr.get_data_fred("DFF", start=start_date, end=end_date) # Federal Funds Rate
vix = pdr.get_data_fred("VIXCLS", start=start_date, end=end_date) # Vix
pce = pdr.get_data_fred("PCE", start=start_date, end=end_date) # Personal consumption
psavert = pdr.get_data_fred("PSAVERT", start=start_date, end=end_date) # Personal savings rate

# Interpolate GDP to monthly 
gdp = pdr.get_data_fred("GDP", start=start_date, end=end_date) # Quarterly GDP
gdp_monthly = gdp.resample("MS").asfreq().interpolate(method='linear')
gdp_change = gdp_monthly.pct_change()

# resampling to month start
cpi = cpi.resample("MS").asfreq()
unrate = unrate.resample("MS").asfreq()
dff = dff.resample("MS").asfreq()
vix = vix.resample("MS").asfreq()
pce = pce.resample("MS").asfreq()
psavert = psavert.resample("MS").asfreq()

# Calculate percent changes
psave_change = psavert.pct_change() # Change in personal savings rate
pconsum_change = pce.pct_change() # % change in personal consumption

# Combining
macro_df = pd.concat([
    psavert,
    gdp_change.rename(columns={'GDP': 'gdp_change'}),
    pconsum_change.rename(columns={'PCE': 'pconsum_change'}),
    psave_change.rename(columns={'PSAVERT': 'psave_change'}),
    cpi.rename(columns={'MEDCPIM158SFRBCLE': 'cpi'}),
    unrate.rename(columns={'UNRATE': 'unrate'}),
    dff.rename(columns={'DFF': 'dff'})
], axis=1)

macro_df.dropna(inplace=True)

In [3]:
macro_df.head()

,PSAVERT,gdp_change,pconsum_change,psave_change,cpi,unrate,dff
DATE,,,,,,,
2015-11-01,5.6,0.001642,0.002283,-0.034483,2.265481,5.1,0.07
2015-12-01,5.8,0.001639,0.002318,0.035714,1.834720,5.0,0.13
2016-01-01,6.1,0.001636,0.002088,0.051724,2.708417,4.8,0.20
2016-02-01,5.6,0.003343,0.006131,-0.081967,2.537714,4.9,0.38
2016-03-01,5.9,0.003331,-0.001394,0.053571,2.222005,5.0,0.36


# Calculating Optimal Stockweight (Labels y)
- What stock weight would have given me the best risk adjusted return this month
- Calculate risk / volatility of returns for the stocks and bonds
- Use actual return and current risk free rate to calculate risk premium
- Use closed form to calculate optimal weight of stock for that month.
- Use as label for the start of month values.

In [4]:
sp500_daily = pdr.get_data_fred('SP500', start=start_date, end=end_date)
bond_daily = pdr.get_data_fred('BAMLCC0A0CMTRIV', start=start_date, end=end_date)

rf = pdr.get_data_fred('SOFR', start='2015-10-01', end='2024-09-30')
rf.columns = ['SOFR']
rf_daily = rf.resample('D').ffill()

index_df = pd.concat([sp500_daily, bond_daily, rf_daily], axis=1).dropna()

In [5]:
index_df.head()

,SP500,BAMLCC0A0CMTRIV,SOFR
DATE,,,
2018-04-03,2614.45,2839.35,1.83
2018-04-04,2644.69,2838.79,1.74
2018-04-05,2662.84,2834.57,1.75
2018-04-06,2604.47,2845.92,1.75
2018-04-09,2613.16,2846.51,1.75


In [6]:
index_df['SP500_return'] = index_df['SP500'].pct_change()
index_df['Bond_return'] = index_df['BAMLCC0A0CMTRIV'].pct_change()

# Resample to monthly
monthly_groups = index_df.resample('ME')

closed_form_weights = []
for date, group in monthly_groups:
    
    # Realized returns
    mu_s = group['SP500_return'].mean()
    mu_b = group['Bond_return'].mean()
    sigma_s = group['SP500_return'].std()
    sigma_b = group['Bond_return'].std()
    rho = group['SP500_return'].corr(group['Bond_return'])
    
    # Monthly risk-free rate
    rf_month = group['SOFR'].iloc[-1] / 100 / 12

    # Excess returns
    mu_s_excess = mu_s - rf_month
    mu_b_excess = mu_b - rf_month

    # Closed-form numerator & denominator
    numerator = mu_s_excess * sigma_b**2 - mu_b_excess * rho * sigma_s * sigma_b
    denominator = (
        mu_s_excess * sigma_b**2 +
        mu_b_excess * sigma_s**2 -
        (mu_s_excess + mu_b_excess) * rho * sigma_s * sigma_b
    )
    if denominator == 0 or np.isnan(denominator):
        w_star = np.nan
    else:
        w_star = numerator / denominator
        w_star = np.clip(w_star, 0, 1)
    closed_form_weights.append({
        'Date': date,
        'optimal_stock_w': w_star
    })
stock_w = pd.DataFrame(closed_form_weights).set_index('Date')
stock_w.dropna(inplace=True)
stock_w.index = stock_w.index - pd.offsets.MonthEnd(1) + pd.offsets.MonthBegin(1)

In [7]:
stock_w.head()

,optimal_stock_w
Date,
2018-04-01,0.038086
2018-05-01,0.176653
2018-06-01,0.206430
2018-07-01,0.000000
2018-08-01,0.094574


# Add stock and bond index data to input matrix

In [8]:
month_starts = macro_df.index

# Getting month start values for S&P500 and bond portfolio returns.
index_at_month_start = index_df.reindex(month_starts, method='ffill')
index_monthly = index_at_month_start[[
    'SP500', 'SP500_return', 'BAMLCC0A0CMTRIV',
    'Bond_return','SOFR'
]]

# Merge w/ macro data to be used for our input layer
df_merged = macro_df.join(index_monthly, how='inner')

# Add monthly return data
monthly_returns = index_df.resample('ME').apply({
    'SP500': lambda x: x.iloc[-1] / x.iloc[0] - 1,
    'BAMLCC0A0CMTRIV': lambda x: x.iloc[-1] / x.iloc[0] - 1
}).rename(columns={
    'SP500': 'SP500_Prev_Month_Return',
    'BAMLCC0A0CMTRIV': 'Bond_Prev_Month_Return'
})
monthly_returns.index = monthly_returns.index + pd.offsets.MonthBegin(1)
df_merged = df_merged.join(monthly_returns, how='inner')
df_merged = df_merged.join(stock_w, how='inner')

In [9]:
print(df_merged.shape)
df_merged.head()

(77, 15)


,PSAVERT,gdp_change,pconsum_change,psave_change,cpi,unrate,dff,SP500,SP500_return,BAMLCC0A0CMTRIV,Bond_return,SOFR,SP500_Prev_Month_Return,Bond_Prev_Month_Return,optimal_stock_w
2018-05-01,6.0,0.003528,0.004813,0.000000,2.981833,3.8,1.70,2654.80,0.002549,2812.07,-0.003335,1.76,0.012852,-0.006294,0.176653
2018-06-01,6.3,0.003515,0.001827,0.050000,2.632118,4.0,1.70,2734.62,0.010849,2822.76,-0.004029,1.81,0.019011,0.007863,0.206430
2018-07-01,6.5,0.003503,0.003187,0.031746,2.111494,3.8,1.91,2718.37,0.000758,2818.58,0.000429,2.12,-0.005942,-0.001481,0.000000
2018-08-01,6.6,0.001909,0.002805,0.015385,2.296711,3.8,1.91,2813.36,-0.001040,2833.24,-0.002299,1.88,0.032853,0.008173,0.094574
2018-09-01,6.8,0.001906,0.000271,0.030303,2.333723,3.7,1.91,2901.52,0.000134,2855.22,-0.000252,1.97,0.031336,0.007758,0.182709


# Preprocessing data and creating tensors

In [10]:
X = df_merged[[col for col in df_merged.columns if col != 'optimal_stock_w']]
y = df_merged['optimal_stock_w']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
df_merged.to_csv('project_data.csv')